In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    #print("nse3 : ", nse3)
    #print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'nak_auto_13'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/nak/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['도개_2013.xlsx', '도개_2014.xlsx', '도개_2015.xlsx', '도개_2016.xlsx', '도개_2017.xlsx', '도개_2018.xlsx', '도개_2019.xlsx', '도개_2020.xlsx'], ['신암_2013.xlsx', '신암_2014.xlsx', '신암_2015.xlsx', '신암_2016.xlsx', '신암_2017.xlsx', '신암_2018.xlsx', '신암_2019.xlsx', '신암_2020.xlsx'], ['풍양_2013.xlsx', '풍양_2014.xlsx', '풍양_2015.xlsx', '풍양_2016.xlsx', '풍양_2017.xlsx', '풍양_2018.xlsx', '풍양_2019.xlsx', '풍양_2020.xlsx']]
data/nak/자동/도개_2013.xlsx
data/nak/자동/도개_2014.xlsx
data/nak/자동/도개_2015.xlsx
data/nak/자동/도개_2016.xlsx
data/nak/자동/도개_2017.xlsx
data/nak/자동/도개_2018.xlsx
data/nak/자동/도개_2019.xlsx
data/nak/자동/도개_2020.xlsx
time range in files :  2013-01-01 00:00  ~  2020-11-15 23:00
data/nak/자동/신암_2013.xlsx
data/nak/자동/신암_2014.xlsx
data/nak/자동/신암_2015.xlsx
data/nak/자동/신암_2016.xlsx
data/nak/자동/신암_2017.xlsx
data/nak/자동/신암_2018.xlsx
data/nak/자동/신암_2019.xlsx
data/nak/자동/신암_2020.xlsx
time range in files :  2013-01-01

1/1 [==============================] - 0s 20ms/step - gen_loss: 14.8452 - disc_loss: 0.3190 - rmse: 0.4372 - val_loss: 0.3984
Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.0051 - disc_loss: 0.3136 - rmse: 0.3810 - val_loss: 0.4004
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.8022 - disc_loss: 0.3165 - rmse: 0.4555 - val_loss: 0.3531
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.2534 - disc_loss: 0.3092 - rmse: 0.3805 - val_loss: 0.4458
Epoch 53/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 10.6159 - disc_loss: 0.3284 - rmse: 0.4271 - val_loss: 0.3253
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.3300 - disc_loss: 0.3043 - rmse: 0.3765 - val_loss: 0.4024
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.7148 - disc_loss: 0.2762 - rmse: 0.3950 - val_loss: 0.3866
Epoch 56/2000
1/1 [=================

Epoch 108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6200 - disc_loss: 0.2260 - rmse: 0.2935 - val_loss: 0.2800
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.3481 - disc_loss: 0.2520 - rmse: 0.3349 - val_loss: 0.2986
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0008 - disc_loss: 0.2450 - rmse: 0.3275 - val_loss: 0.2903
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1942 - disc_loss: 0.2684 - rmse: 0.3077 - val_loss: 0.3558
Epoch 112/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 6.1479 - disc_loss: 0.2403 - rmse: 0.2986 - val_loss: 0.3451
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0733 - disc_loss: 0.2458 - rmse: 0.3157 - val_loss: 0.3194
Epoch 114/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8798 - disc_loss: 0.2538 - rmse: 0.3352 - val_loss: 0.3081
Epoch 115/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6180 - disc_loss: 0.2394 - rmse: 0.2963 - val_loss: 0.2994
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9240 - disc_loss: 0.2290 - rmse: 0.3381 - val_loss: 0.2981
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3890 - disc_loss: 0.2308 - rmse: 0.3299 - val_loss: 0.2707
Epoch 169/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.1947 - disc_loss: 0.2469 - rmse: 0.3325 - val_loss: 0.3144
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5652 - disc_loss: 0.2196 - rmse: 0.3179 - val_loss: 0.2773
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0414 - disc_loss: 0.2244 - rmse: 0.3075 - val_loss: 0.3004
Epoch 172/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0842 - disc_loss: 0.2251 - rmse: 0.3652 - val_loss: 0.3011
Epoch 173/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0616 - disc_loss: 0.2281 - rmse: 0.2819 - val_loss: 0.3141
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6135 - disc_loss: 0.2391 - rmse: 0.2817 - val_loss: 0.2710
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7187 - disc_loss: 0.2238 - rmse: 0.3096 - val_loss: 0.2788
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6808 - disc_loss: 0.2365 - rmse: 0.2597 - val_loss: 0.2716
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.8860 - disc_loss: 0.2356 - rmse: 0.3025 - val_loss: 0.2688
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7904 - disc_loss: 0.2018 - rmse: 0.2433 - val_loss: 0.3303
Epoch 230/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2425 - disc_loss: 0.2295 - rmse: 0.2997 - val_loss: 0.2895
Epoch 231/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4650 - disc_loss: 0.1600 - rmse: 0.2569 - val_loss: 0.2455
Epoch 1733/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3366 - disc_loss: 0.1580 - rmse: 0.2465 - val_loss: 0.2529
Epoch 1734/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2437 - disc_loss: 0.1571 - rmse: 0.2390 - val_loss: 0.3282
Epoch 1735/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.9916 - disc_loss: 0.1514 - rmse: 0.2174 - val_loss: 0.2426
Epoch 1736/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5079 - disc_loss: 0.1622 - rmse: 0.2859 - val_loss: 0.2334
Epoch 1737/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.0832 - disc_loss: 0.1564 - rmse: 0.2322 - val_loss: 0.2223
Epoch 1738/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7159 - disc_loss: 0.1516 - rmse: 0.2202 - val_loss: 0.2084
Epoch 1739/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3395 - disc_loss: 0.1481 - rmse: 0.2555 - val_loss: 0.2615
Epoch 1791/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.4527 - disc_loss: 0.1613 - rmse: 0.2549 - val_loss: 0.2341
Epoch 1792/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6550 - disc_loss: 0.1601 - rmse: 0.2718 - val_loss: 0.2475
Epoch 1793/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4537 - disc_loss: 0.1486 - rmse: 0.2513 - val_loss: 0.3603
Epoch 1794/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.1667 - disc_loss: 0.1521 - rmse: 0.2436 - val_loss: 0.3017
Epoch 1795/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1824 - disc_loss: 0.1641 - rmse: 0.2441 - val_loss: 0.3222
Epoch 1796/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9537 - disc_loss: 0.1583 - rmse: 0.2170 - val_loss: 0.2620
Epoch 1797/2000
1/1 [========

1/1 [==============================] - 0s 21ms/step - gen_loss: 3.6678 - disc_loss: 0.1494 - rmse: 0.2454 - val_loss: 0.2532
Epoch 1849/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.4178 - disc_loss: 0.1463 - rmse: 0.2433 - val_loss: 0.2262
Epoch 1850/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.5432 - disc_loss: 0.1619 - rmse: 0.2430 - val_loss: 0.3128
Epoch 1851/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6306 - disc_loss: 0.1539 - rmse: 0.2189 - val_loss: 0.2554
Epoch 1852/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5838 - disc_loss: 0.1523 - rmse: 0.2595 - val_loss: 0.2339
Epoch 1853/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0270 - disc_loss: 0.1634 - rmse: 0.2303 - val_loss: 0.2543
Epoch 1854/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1085 - disc_loss: 0.1576 - rmse: 0.2416 - val_loss: 0.2399
Epoch 1855/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7297 - disc_loss: 0.1407 - rmse: 0.2195 - val_loss: 0.2480
Epoch 1907/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.5756 - disc_loss: 0.1364 - rmse: 0.2087 - val_loss: 0.3062
Epoch 1908/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0066 - disc_loss: 0.1410 - rmse: 0.2372 - val_loss: 0.2158
Epoch 1909/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8393 - disc_loss: 0.1557 - rmse: 0.2492 - val_loss: 0.2209
Epoch 1910/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9677 - disc_loss: 0.1534 - rmse: 0.2681 - val_loss: 0.2513
Epoch 1911/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1778 - disc_loss: 0.1510 - rmse: 0.2740 - val_loss: 0.3875
Epoch 1912/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8746 - disc_loss: 0.1528 - rmse: 0.2471 - val_loss: 0.2433
Epoch 1913/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7842 - disc_loss: 0.1467 - rmse: 0.2212 - val_loss: 0.2421
Epoch 1965/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8705 - disc_loss: 0.1572 - rmse: 0.3641 - val_loss: 0.2578
Epoch 1966/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8262 - disc_loss: 0.1616 - rmse: 0.2218 - val_loss: 0.2482
Epoch 1967/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1037 - disc_loss: 0.1508 - rmse: 0.2365 - val_loss: 0.3556
Epoch 1968/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4937 - disc_loss: 0.1484 - rmse: 0.2073 - val_loss: 0.2268
Epoch 1969/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5002 - disc_loss: 0.1630 - rmse: 0.2091 - val_loss: 0.2339
Epoch 1970/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0652 - disc_loss: 0.1484 - rmse: 0.2264 - val_loss: 0.2325
Epoch 1971/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [16]:
print(real_df_all.shape)
rnn_target_column = 'do'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(69024, 36)
do


In [17]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (48316, 36) val_df.shape :  (13805, 36) test_df.shape: (6903, 36)


In [18]:
#fake_df.shape[0]/8760

In [19]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [22]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  2
out_num_features :  1


In [23]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [24]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    #if not os.path.exists('save/' + watershed):
    #    os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/nak/models/nak_auto_13/do/


In [25]:
#"save/" + watershed + "models/" + file_parameters['watershed'] + '/' + pa[indices[target_col_idx]]
#model_path

In [26]:
rnn_epochs

20

## 모델 학습

In [27]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=50,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 0.7507 - mean_absolute_error: 0.6791 - nse: 0.3025
Epoch 00001: val_nse improved from -inf to -0.16218, saving model to save/best_model.h5
10/10 [==============================] - 10s 1s/step - loss: 0.7507 - mean_absolute_error: 0.6791 - nse: 0.3025 - val_loss: 0.7348 - val_mean_absolute_error: 0.6765 - val_nse: -0.1622
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 0.3855 - mean_absolute_error: 0.4853 - nse: 0.6564
Epoch 00002: val_nse improved from -0.16218 to 0.08290, saving model to save/best_model.h5
10/10 [==============================] - 10s 996ms/step - loss: 0.3855 - mean_absolute_error: 0.4853 - nse: 0.6564 - val_loss: 0.5801 - val_mean_absolute_error: 0.5990 - val_nse: 0.0829
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 0.3172 - mean_absolute_error: 0.4331 - nse: 0.7083
Epoch 00003: val_nse improved from 0.08290 to 0.38335, saving model to save/best_model.h5
10/1

Epoch 22/50
10/10 [==============================] - ETA: 0s - loss: 0.2084 - mean_absolute_error: 0.3385 - nse: 0.8168
Epoch 00022: val_nse did not improve from 0.50903
10/10 [==============================] - 10s 974ms/step - loss: 0.2084 - mean_absolute_error: 0.3385 - nse: 0.8168 - val_loss: 0.5465 - val_mean_absolute_error: 0.5470 - val_nse: 0.1380
Epoch 23/50
10/10 [==============================] - ETA: 0s - loss: 0.2103 - mean_absolute_error: 0.3402 - nse: 0.8108
Epoch 00023: val_nse did not improve from 0.50903
10/10 [==============================] - 10s 955ms/step - loss: 0.2103 - mean_absolute_error: 0.3402 - nse: 0.8108 - val_loss: 0.4754 - val_mean_absolute_error: 0.5188 - val_nse: 0.2516
Epoch 24/50
10/10 [==============================] - ETA: 0s - loss: 0.2012 - mean_absolute_error: 0.3337 - nse: 0.8181
Epoch 00024: val_nse did not improve from 0.50903
10/10 [==============================] - 10s 965ms/step - loss: 0.2012 - mean_absolute_error: 0.3337 - nse: 0.8181 - v

Epoch 45/50
10/10 [==============================] - ETA: 0s - loss: 0.1498 - mean_absolute_error: 0.2918 - nse: 0.8677
Epoch 00045: val_nse did not improve from 0.50903
10/10 [==============================] - 9s 948ms/step - loss: 0.1498 - mean_absolute_error: 0.2918 - nse: 0.8677 - val_loss: 0.8647 - val_mean_absolute_error: 0.7101 - val_nse: -0.3652
Epoch 46/50
10/10 [==============================] - ETA: 0s - loss: 0.1466 - mean_absolute_error: 0.2848 - nse: 0.8728
Epoch 00046: val_nse did not improve from 0.50903
10/10 [==============================] - 10s 970ms/step - loss: 0.1466 - mean_absolute_error: 0.2848 - nse: 0.8728 - val_loss: 0.7692 - val_mean_absolute_error: 0.6691 - val_nse: -0.2135
Epoch 47/50
10/10 [==============================] - ETA: 0s - loss: 0.1524 - mean_absolute_error: 0.2899 - nse: 0.8611
Epoch 00047: val_nse did not improve from 0.50903
10/10 [==============================] - 10s 966ms/step - loss: 0.1524 - mean_absolute_error: 0.2899 - nse: 0.8611 - 

## core / window.py / 

In [34]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",df_length)
print(teg_check)
print('nse_val : ', val_nse['GRU'])
#print("---------------")
#print(val_nse['GRU'])
#print(val_pbias['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  9.796826
predict_day :  5
------------------------
0.8214287 1.192529 1.0920297
save/nak/models/nak_auto_13/do/
year : 2013 ~ 2020
run :  range(0, 1)
target :  do   2
length :  69024
train=7, test=1, val=2
nse_val :  [0.3913358]


In [35]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print('nse_all : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  10.354572
predict_day :  5
------------------------
0.70503706 0.9296699 0.96419394
nse_all :  [0.71857727]


In [36]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('nse_train : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  10.370092
predict_day :  5
------------------------
0.6662998 0.8246356 0.9080945
nse_train :  [0.7563266]


In [37]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('nse_test : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  11.367396
predict_day :  5
------------------------
0.7276854 1.0464371 1.0229551
nse_test :  [0.76102024]


In [32]:
#train_df.shape, val_df.shape, test_df.shape

In [33]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column, " ", out_features[0])
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
